In [1]:
# Importações
from bs4 import BeautifulSoup
from selenium import webdriver
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from docx import Document
from docx.shared import Inches
from time import sleep
from datetime import date, datetime
import requests
import pandas as pd
import numpy as np
import lxml
import json

In [11]:
# Definição de Configurações
headers = {  # <-- so the Google will treat your script as a "real" user browser.
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

In [12]:
# Execução de Browser e definição de configurações
driver = webdriver.Chrome()
driver.set_window_size(1440,1000)
# driver.get('https://www.google.com/aclk?sa=L&ai=DChcSEwjD4MTr367yAhUMiJEKHW__C8YYABAlGgJjZQ&sig=AOD64_10iTpfJl-Fh12ZyJ0WTGMGfr5arQ&ctype=5&q=&ved=0ahUKEwiX6cDr367yAhXbqJUCHdV7A0YQg-UECPgE&adurl=')
# driver.save_screenshot("screenshot.png")

In [14]:
# Pesquisa de preços
material = pd.read_excel('pesquisa.xlsx')

data_mat = []
items_busca = []

for i in range(len(material)):
    mat_item = {}
    peca = material.loc[i, "material"]
    items_busca.append(peca)
    mat = f"https://www.google.com/search?q={peca.replace(' ', '+')}&tbm=shop"
    response = requests.get(mat, headers=headers).text
    soup = BeautifulSoup(response, 'lxml')
#     material_busca = soup.find('h4', class_='Xjkr3b').text
    material_busca = soup.find('span', class_='C7Lkve')
    mat_busca = f"https://www.google.com/search?q={peca.replace(' ', '+')}&tbm=shop"
    response_busca = requests.get(mat, headers=headers).text
    soup_busca = BeautifulSoup(response, 'lxml')
    print(response)
    for n, container in enumerate(soup_busca.findAll('div', class_='sh-dgr__content')[0:5]):
        title = container.find('h4', class_='Xjkr3b').text
        price = container.find('span', class_='a8Pemb OFFNJ').text
        supplier = container.find('div', class_='aULzUe IuHnof').text
        link = container.find('a', class_='eaGTj mQaFGe shntl')
#         driver.get(f"https://www.google.com{link['href']}")
#         sleep(3)
#         driver.save_screenshot(f"img_buscas/{title}_{supplier}.png")
#         img = f'img_buscas/{title}_{supplier}.png'
        if supplier == 'Mercado Livre':
            response_ml = requests.get(f"https://www.google.com{link['href']}", headers=headers).text
            soup_ml = BeautifulSoup(response_ml, 'lxml')
            supplier_ml = soup_ml.find('a', class_='ui-pdp-media__action ui-box-component__action')['href']
            response_ml_2 = requests.get(supplier_ml, headers=headers).text
            soup_ml_2 = BeautifulSoup(response_ml_2, 'lxml')
            supplier = f"{soup_ml_2.find('h3', class_='store-info__name').text} (Mercado Livre)"
        item = [
            ('nome', title),
            ('fornecedor', supplier),
            ('valor', price),
            ('link', f"https://www.google.com{link['href']}"),
#             ('img', img)
        ]
        for field in item:
            mat_item[f'{field[0]}_busca_{n+1}'] = field[1]
    data_mat.append(mat_item)

<!doctype html><html lang="pt-BR"><head><title>Smart tv 32 polegadas samsumg - Google Shopping</title><meta content="width=device-width,maximum-scale=1.0,initial-scale=1.0,minimum-scale=0.9,user-scalable=no" name="viewport"><meta content="address=no" name="format-detection"><meta content="email=no" name="format-detection"><meta content="telephone=no" name="format-detection"><meta content="origin" name="referrer"><script nonce="dGPa+AKLXLj717qcTtKfuQ==">(function(){
var a=window.performance;window.start=Date.now();a:{var b=window;if(a){var c=a.timing;if(c){var d=c.navigationStart,f=c.responseStart;if(f>d&&f<=window.start){window.start=f;b.wsrt=f-d;break a}}a.now&&(b.wsrt=Math.floor(a.now()))}}window.google=window.google||{};}).call(this);</script><style>a{color:#1a73e8;text-decoration:none}a img{border:none}body{margin:0;}html{background:#fff;font-family:Roboto,'Helvetica Neue',Arial,sans-serif;font-size:14px;color:#202124;line-height:20px;word-wrap:break-word}</style><style>body.noscro

<!doctype html><html lang="pt-BR"><head><title>Smartphone Motorola - Google Shopping</title><meta content="width=device-width,maximum-scale=1.0,initial-scale=1.0,minimum-scale=0.9,user-scalable=no" name="viewport"><meta content="address=no" name="format-detection"><meta content="email=no" name="format-detection"><meta content="telephone=no" name="format-detection"><meta content="origin" name="referrer"><script nonce="FaBVWKkqWg6JKVdbJfiOOQ==">(function(){
var a=window.performance;window.start=Date.now();a:{var b=window;if(a){var c=a.timing;if(c){var d=c.navigationStart,f=c.responseStart;if(f>d&&f<=window.start){window.start=f;b.wsrt=f-d;break a}}a.now&&(b.wsrt=Math.floor(a.now()))}}window.google=window.google||{};}).call(this);</script><style>a{color:#1a73e8;text-decoration:none}a img{border:none}body{margin:0;}html{background:#fff;font-family:Roboto,'Helvetica Neue',Arial,sans-serif;font-size:14px;color:#202124;line-height:20px;word-wrap:break-word}</style><style>body.noscroll{overflo

In [ ]:
df = pd.DataFrame(data_mat, index=items_busca)
df

In [ ]:
data_mat

https://roytuts.com/add-images-to-word-file-using-python/
https://stackoverflow.com/questions/27327513/create-pdf-from-a-list-of-images
https://pythonspot.com/selenium-take-screenshot/


In [ ]:
# Criar planilha com resultados
wb = Workbook()

ws = wb.active

# ws1 = wb.create_sheet("Mysheet")

ws.title = "New Title"

for r in dataframe_to_rows(df, index=True, header=True):
    ws.append(r)

wb.save('testes.xlsx')

In [ ]:
for n, i in enumerate(data_mat):
    print(i[f'nome_busca_{1}'], '|', i[f'fornecedor_busca_{1}'], '|', i[f'valor_busca_{1}'], '|', i[f'link_busca_{1}'])
    print(i[f'nome_busca_{2}'], '|', i[f'fornecedor_busca_{2}'], '|', i[f'valor_busca_{1}'], '|', i[f'link_busca_{1}'])
    print(i[f'nome_busca_{3}'], '|', i[f'fornecedor_busca_{3}'], '|', i[f'valor_busca_{1}'], '|', i[f'link_busca_{1}'])
    print(i[f'nome_busca_{4}'], '|', i[f'fornecedor_busca_{4}'], '|', i[f'valor_busca_{1}'], '|', i[f'link_busca_{1}'])
    print(i[f'nome_busca_{5}'], '|', i[f'fornecedor_busca_{5}'], '|', i[f'valor_busca_{1}'], '|', i[f'link_busca_{1}'])
    print('')

In [ ]:
materiais_result = []
for mat in data_mat:
    items = []
    for key in mat:
        items.append(mat[key])
    materiais_result.append(items)
print(materiais_result)

In [ ]:
for i in materiais_result:
    print(i[3])

In [ ]:
data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M')
data_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y')
datetime_format = data_e_hora_atuais.strftime('%d-%m-%Y-%H-%M')
hora_em_texto = data_e_hora_atuais.strftime('%H:%M')

print(data_e_hora_em_texto)
print(data_em_texto)
print(hora_em_texto)
print(datetime_format)

In [ ]:
# Criar documento Word com resultados
document = Document()

document.add_heading(f'Pesquisa de Preços - {data_e_hora_em_texto}', 0)

materiais_busca_2 = []
for mat in data_mat:
    items = []
    for key in mat:
        items.append(mat[key])
    materiais_busca_2.append(items)

for n, i in enumerate(materiais_busca_2):
    h = document.add_heading(f'Item pesquisado: {items_busca[n]}', level=1)
    hr = h.add_run()
    hr.add_break()
    hr.add_text('==============================')
    document.add_heading(f'Fornecedor 1: {i[1]}', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text(f"Descrição: {i[0]}")
    r.add_break()
    r.add_text(f"Valor: {i[2]}")
    r.add_break()
    r.add_text(f"Endereço: {i[3]}")
    r.add_break()
    driver.get(i[3])
    sleep(5)
    driver.save_screenshot(f"busca_{n+1}_{i[1]}.png")
    document.add_picture(f"busca_{n+1}_{i[1]}.png", width=Inches(6))
    document.add_heading('_____________________________________________________________')
    document.add_paragraph()
    document.add_page_break()
    
    document.add_heading(f'Fornecedor 2: {i[5]}', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text(f"Descrição: {i[4]}")
    r.add_break()
    r.add_text(f"Valor: {i[6]}")
    r.add_break()
    r.add_text(f"Link: {i[7]}")
    r.add_break()
    driver.get(i[7])
    sleep(5)
    driver.save_screenshot(f"busca_{n+1}_{i[1]}.png")
    document.add_picture(f"busca_{n+1}_{i[1]}.png", width=Inches(6))
    document.add_heading('_____________________________________________________________')
    document.add_paragraph()
    document.add_page_break()
    
    document.add_heading(f'Fornecedor 3: {i[9]}', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text(f"Descrição: {i[8]}")
    r.add_break()
    r.add_text(f"Valor: {i[10]}")
    r.add_break()
    r.add_text(f"Link: {i[11]}")
    r.add_break()
    driver.get(i[11])
    sleep(5)
    driver.save_screenshot(f"busca_{n+1}_{i[1]}.png")
    document.add_picture(f"busca_{n+1}_{i[1]}.png", width=Inches(6))
    document.add_heading('_____________________________________________________________')
    document.add_paragraph()
    document.add_page_break()
    
    document.add_heading(f'Fornecedor 4: {i[13]}', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text(f"Descrição: {i[12]}")
    r.add_break()
    r.add_text(f"Valor: {i[14]}")
    r.add_break()
    r.add_text(f"Link: {i[15]}")
    r.add_break()
    driver.get(i[15])
    sleep(5)
    driver.save_screenshot(f"busca_{n+1}_{i[1]}.png")
    document.add_picture(f"busca_{n+1}_{i[1]}.png", width=Inches(6))
    document.add_heading('_____________________________________________________________')
    document.add_paragraph()
    document.add_page_break()
    
    document.add_heading(f'Fornecedor 5: {i[17]}', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text(f"Descrição: {i[16]}")
    r.add_break()
    r.add_text(f"Valor: {i[18]}")
    r.add_break()
    r.add_text(f"Link: {i[19]}")
    r.add_break()
    driver.get(i[19])
    sleep(5)
    driver.save_screenshot(f"busca_{n+1}_{i[1]}.png")
    document.add_picture(f"busca_{n+1}_{i[1]}.png", width=Inches(6))
    document.add_heading('_____________________________________________________________')
    document.add_paragraph()
    document.add_page_break()
    
document.save(f'pesquisa-{datetime_format}.docx')